# InterTwin 4th Technical Meeting

13/06/2024, Vienna, AT

### WP6: Example CWL workflow execution

This notebook demonstrates how to execute a CWL workflow using the `cwltool` command line tool. The notebook showcases all of the different parts which make up the OGC Application Package to execute a CWL workflow.

This notebook is showcasing the drought forecasting use case from the InterTwin project. This Application Package is based on `HydroMT` developed by Deltares.

#### Table of Contents

1. [Section 1: Dependencies](#section-1-dependencies)
2. [Section 2: Dockerfile](#section-2-packaging-necessary-files)
3. [Section 3: OGC Application Package](#section-3-ogc-application-package)

#### Section 1: Dependencies

Local development and  execution specific dependencies:

- `cwltool` - A reference implementation of the Common Workflow Language.
- `docker` - A tool designed to make it easier to create, deploy, and run applications by using containers.
- `conda` - An open-source package management system and environment management system that runs on Windows, macOS, and Linux.

Clone this repository:

```bash
git clone https://github.com/jzvolensky/Itwin-tech-meeting.git
```

Install the conda environment:

```bash
conda env create -f environment.yml
```

Activate the conda environment:

```bash
conda activate itwin-tech-meeting
```


#### Section 2: Packaging Necessary files

There are several files which we need to include in our Docker image so that during the execution of the CWL workflow, the necessary files are available. These can be found in the `/example` directory.


`data_catalog.yaml` - This file contains the metadata for the input data files. This file is used by the `HydroMT` package to read the input data files.
`wflow.ini` - Configuration file for the `HydroMT` package.
`requirements.txt` - Python package requirements for the `HydroMT` package.
`build.sh` - A shell script to build the `HydroMT` model.
`config_gen.py` - A Python script to generate the configuration file for the `HydroMT` model based on workflow inputs.

Docker Image Structure:

```bash
/hydromt
    /requirements.txt
    /wflow.ini
    /data_catalog.yaml
    /config_gen.py
    /output/
    /data/
```

The resulting Docker image is published on DockerHub.

If you want to build and publish it yourself, you can use the following command:

```bash
docker login
docker build -t image_name .
docker tag image_name username/image_name:tag
docker push username/image_name:tag
```



In [ ]:
!docker login

In [ ]:
!docker build -t hydromt . 

In [ ]:
!docker tag hydromt:latest potato55/hydromt:vienna

In [ ]:
!docker push potato55/hydromt:vienna

#### Section 3: OGC Application Package

In the `/example/cwl` directory, you will find the CWL workflows necessary to execute `HydroMT`. The directory contains three files:

`hydromt-build.cwl` - This CWL workflow builds the `HydroMT` model.
`update_config.cwl` - This is a subworkflow which updates the configuration file for the `HydroMT` model before it is built.
`params.yaml` - This file contains the input parameters for the CWL workflow.

To execute the CWL workflow, you can use the following command:

```bash
cwltool --verbose --force-docker-pull --no-read-only --outdir ./hydromt-output hydromt-build.cwl#hydromt-build params.yaml
```

There are several flags which are used in the command:

- `--verbose` - This flag outputs more information about the execution of the CWL workflow.
- `--force-docker-pull` - This flag forces the Docker image to be pulled from the DockerHub repository. (This is not always necessary, we just want the latest version).
- `--no-read-only` - This flag ensures that the Docker container is not read-only (Although this is not always the case).
- `--outdir` - This flag specifies the output directory for the CWL workflow on our local machine.

The CWL is also defined as `hydromt-build.cwl#hydromt-build` which specifies the CWL workflow to execute. The `params.yaml` file contains the input parameters for the CWL workflow.

Try it out below!

In [8]:
!cwltool --verbose --no-read-only --force-docker-pull --outdir ./hydromt-output ./example/hydromt/cwl/hydromt-build-workflow.cwl#hydromt-workflow ./example/hydromt/cwl/params.yaml

INFO /home/jzvolensky/miniconda3/envs/itwin-cwl/bin/cwltool 3.1.20240404144621
INFO Resolved './example/hydromt/cwl/hydromt-build-workflow.cwl#hydromt-workflow' to 'file:///home/jzvolensky/eurac/projects/Itwin-tech-meeting/example/hydromt/cwl/hydromt-build-workflow.cwl#hydromt-workflow'
INFO [workflow ] start
INFO [workflow ] starting step update-config
INFO [step update-config] start
INFO ['docker', 'pull', 'potato55/hydromt:vienna']
vienna: Pulling from potato55/hydromt
Digest: sha256:6771b1cd17e977210c5558e22d86d79df80169e9138f1bf495f253063922d7bd
Status: Image is up to date for potato55/hydromt:vienna
docker.io/potato55/hydromt:vienna
INFO [job update-config] /tmp/8ewr601g$ docker \
    run \
    -i \
    --mount=type=bind,source=/tmp/8ewr601g,target=/hydromt \
    --mount=type=bind,source=/tmp/dln_3lbi,target=/tmp \
    --workdir=/hydromt \
    --user=1149421890:1149400513 \
    --rm \
    --cidfile=/tmp/wkoio_ef/20240611181200-426473.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/

In [7]:
!cwltool --outdir ./wflow-output --no-read-only --no-match-user ./example/wflow/cwl/wflow-run.cwl#run-wflow ./example/wflow/cwl/params-wflow.yaml

INFO /home/jzvolensky/miniconda3/envs/itwin-cwl/bin/cwltool 3.1.20240404144621
INFO Resolved './example/wflow/cwl/wflow-run.cwl#run-wflow' to 'file:///home/jzvolensky/eurac/projects/Itwin-tech-meeting/example/wflow/cwl/wflow-run.cwl#run-wflow'
example/wflow/cwl/wflow-run.cwl:58:15: object id 'example/wflow/cwl/wflow-run.cwl#wflow-workflow/chmod_step/run/netcdf_output' previously defined
WARNING example/wflow/cwl/wflow-run.cwl:58:15: object id 'example/wflow/cwl/wflow-run.cwl#wflow-workflow/chmod_step/run/netcdf_output' previously defined
example/wflow/cwl/wflow-run.cwl:49:13: object id 'example/wflow/cwl/wflow-run.cwl#wflow-workflow/chmod_step/netcdf_output' previously defined
WARNING example/wflow/cwl/wflow-run.cwl:49:13: object id 'example/wflow/cwl/wflow-run.cwl#wflow-workflow/chmod_step/netcdf_output' previously defined
example/wflow/cwl/wflow-run.cwl:58:15: object id 'example/wflow/cwl/wflow-run.cwl#wflow-workflow/chmod_step/run/netcdf_output' previously defined
WARNING example/wf